In [1]:
#import pyspark
#sc = pyspark.SparkContext.getOrCreate()
#sqlContext = pyspark.sql.SQLContext(sc)
from pyspark.sql import Row
from pyspark.sql.functions import *
import pandas as pd # also need "xlrd" module in our system
import xlrd

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
xl_workbook = xlrd.open_workbook("lib-statistics.xlsx")
#sheet_names = xl_workbook.sheet_names()
#print('Sheet Names', sheet_names)
#xl_sheet = xl_workbook.sheet_by_name(sheet_names[0])
xl_sheet = xl_workbook.sheet_by_index(0)
#print ('Sheet name: %s' % xl_sheet.name)
data = []
for row in range(xl_sheet.nrows):
    line = []
    for col in range(xl_sheet.ncols):
        line.append(str(xl_sheet.cell(row, col).value))
    data.append(line)
#print((data))

In [4]:
columnNames = data[0]
dataValues = data[1:]
dataPairs = []
for row in range(len(dataValues)):
    line = {}
    for col in range(len(columnNames)):
        line[columnNames[col]] = dataValues[row][col]
    dataPairs.append(Row(**line))
#dataPairs

In [5]:
dataRDD = sc.parallelize(dataPairs)
df = dataRDD.toDF()
df.show()

+---+--------------------+--------------------+-----------------+--------------------+------------+
|Dil|            Eser Adı|     Konu Başlıkları|        Sınıflama|               Yazar|Ödünç Sayısı|
+---+--------------------+--------------------+-----------------+--------------------+------------+
|eng|Museum frictions ...|Museums--Social A...|      AM 7/.M8713|                    |            |
|tur|Tahayyül gücünü y...|Imagination (Phil...|   B 105 .I49/T34|                    |            |
|tur|Bir yol var : Min...|Ontology
Stress M...|  B 3279 .H48/D36|        Damcı, Taner|            |
|tur|Psikoloji şerhi =...|Aristotle
Aristot...|  B 415 .A9/A7519|            İbn Rüşd|         2.0|
|eng|An examination of...|Özel, İsmet, 1944...| B 5074 .S544/K39|      Kaya, Vefa Can|            |
|eng|An examination of...|Özel, İsmet, 1944...| B 5074 .S544/K39|      Kaya, Vefa Can|            |
|tur|Ruhun uyanışı, ya...|Philosophy, Islam...| B 753 .I53/R5320|İbn Ṭufeyl, Muḥam...|            |


In [6]:
df.printSchema()

root
 |-- Dil: string (nullable = true)
 |-- Eser Adı: string (nullable = true)
 |-- Konu Başlıkları: string (nullable = true)
 |-- Sınıflama: string (nullable = true)
 |-- Yazar: string (nullable = true)
 |-- Ödünç Sayısı: string (nullable = true)



In [7]:
df.filter(df['Konu Başlıkları']=='').collect()

[Row(Dil='tur', Eser Adı='Edebiyat tarihi yazıları : arşiv kayıtları, yazma eserler ve kayıp metinler', Konu Başlıkları='', Sınıflama='PL 216 .E777/E34', Yazar='Erünsal, İsmail E.', Ödünç Sayısı='1.0')]

In [8]:
def countEmptyAndNull(df):
    for i in df.columns:
        print("Column '%s' has %d empty, %d not empty, and %d null rows." % (i, df.filter(df[i]=='').count(), df.filter(df[i]!='').count(), df.filter(df[i].isNull()).count()))


In [9]:
countEmptyAndNull(df)

Column 'Dil' has 0 empty, 751 not empty, and 0 null rows.
Column 'Eser Adı' has 0 empty, 751 not empty, and 0 null rows.
Column 'Konu Başlıkları' has 1 empty, 750 not empty, and 0 null rows.
Column 'Sınıflama' has 0 empty, 751 not empty, and 0 null rows.
Column 'Yazar' has 294 empty, 457 not empty, and 0 null rows.
Column 'Ödünç Sayısı' has 598 empty, 153 not empty, and 0 null rows.


In [10]:
df.groupBy('Dil').count().show()

+---+-----+
|Dil|count|
+---+-----+
|fre|   18|
|tuk|    2|
|geo|    1|
|Tur|    8|
|ara|    4|
|eng|   98|
|xxx|    2|
|tur|  575|
|ota|   38|
|spa|    1|
|chi|    1|
|ger|    3|
+---+-----+



In [11]:
df.count()

751

In [12]:
df2 = df.groupBy('Dil').count()
df2.select(sum(df2['count'])).show()

+----------+
|sum(count)|
+----------+
|       751|
+----------+



In [13]:
df1 = df.select(df['Sınıflama'], df['Eser Adı'], df['Yazar'], df['Dil'], df['Konu Başlıkları'], df['Ödünç Sayısı'].cast('float'))
df1.select(sum(df1['Ödünç Sayısı'])).show()

+-----------------+
|sum(Ödünç Sayısı)|
+-----------------+
|            339.0|
+-----------------+



In [14]:
df1.filter(df1['Ödünç Sayısı'].isNull()).count()

598

In [15]:
countEmptyAndNull(df1)

Column 'Sınıflama' has 0 empty, 751 not empty, and 0 null rows.
Column 'Eser Adı' has 0 empty, 751 not empty, and 0 null rows.
Column 'Yazar' has 294 empty, 457 not empty, and 0 null rows.
Column 'Dil' has 0 empty, 751 not empty, and 0 null rows.
Column 'Konu Başlıkları' has 1 empty, 750 not empty, and 0 null rows.
Column 'Ödünç Sayısı' has 0 empty, 0 not empty, and 598 null rows.


In [16]:
df.groupBy(df['Sınıflama']).count().show()

+------------------+-----+
|         Sınıflama|count|
+------------------+-----+
|   BF 635 .G43/C34|    1|
|       D 16/.H3519|    1|
|   PL 248 .T67/D47|    1|
| BP 189.7 .C58/O57|    1|
|    NK 3636.5/.L57|    1|
|     PN 1997/.F357|    1|
|   DR 479 .E85/P68|    1|
|HQ 1726.7 .A37/T67|    1|
| HQ 767.9 .C36/R47|    1|
|  NA 1363 .K83/K66|    1|
|   PN 6757 .K54/C3|    1|
|     BP 63 .T8/B37|    1|
|       HV 42/.B676|    5|
|        Q 335/.R86|    1|
|   BP 130.1/.M3719|    1|
| DR 438.5 .A36/F45|    1|
|   PL 248 .Y55/H35|    1|
| BF 692.2 .S65/B36|    1|
|        ND 867/.T9|    1|
|    GE 160 .T9/T86|    1|
+------------------+-----+
only showing top 20 rows



In [17]:
df.groupBy(df['Sınıflama']).count().count()

614

In [18]:
df_s = df.groupBy(df['Sınıflama']).count()
print(df_s.filter(df_s['count'] > 1).count())
print(df_s.filter(df_s['count'] < 1).count())
print(df_s.filter(df_s['count'] == 1).count())

43
0
571


In [19]:
df_s.filter(df_s['Sınıflama'].like('%D%')).show()

+------------------+-----+
|         Sınıflama|count|
+------------------+-----+
|       D 16/.H3519|    1|
|   PL 248 .T67/D47|    1|
|   DR 479 .E85/P68|    1|
| DR 438.5 .A36/F45|    1|
|        ND 867/.T9|    1|
|QH 366.2 .D379/B58|    1|
|     BP 161.2/.D56|    1|
|      H 49 .T9/D46|    1|
|   DR 486 .D34/B55|    1|
|   BP 178 .E73/D67|    1|
|       DR 531/.A46|    1|
|        DS 47/.D93|    1|
|       BP 170/.D64|    1|
|       D 20/.H5719|    1|
|        DR 401/.T6|    1|
|   DF 901 .T75/A94|    1|
|   DR 592 .U43/B45|    1|
|       BP 166/.D56|    1|
| DD 867.5 .T87/B47|    1|
|       DR 401/.T84|    2|
+------------------+-----+
only showing top 20 rows



In [20]:
df_s.filter(split(df_s['Sınıflama'], " ")[0].like("%D%")).show()

+-------------------+-----+
|          Sınıflama|count|
+-------------------+-----+
|        D 16/.H3519|    1|
|    DR 479 .E85/P68|    1|
|  DR 438.5 .A36/F45|    1|
|         ND 867/.T9|    1|
|    DR 486 .D34/B55|    1|
|        DR 531/.A46|    1|
|         DS 47/.D93|    1|
|        D 20/.H5719|    1|
|         DR 401/.T6|    1|
|    DF 901 .T75/A94|    1|
|    DR 592 .U43/B45|    1|
|  DD 867.5 .T87/B47|    1|
|        DR 401/.T84|    2|
|          DS 26/.E7|    2|
|   DT 197.5 .I8/M34|    1|
|    HD 1165 .T8/C57|    1|
|    DR 479 .A75/S46|    2|
|    DR 448 .A46/O86|    1|
|      DR 719/.I8836|    1|
|DS 109.32 .M33/U387|    1|
+-------------------+-----+
only showing top 20 rows



In [21]:
df_s.groupBy(split(df_s['Sınıflama'], " ")[0]).count().show()

+----------------------+-----+
|split(Sınıflama,  )[0]|count|
+----------------------+-----+
|                     K|    3|
|                    UA|    1|
|                    LA|    4|
|                    BS|    3|
|                    NK|    2|
|                    PL|   81|
|                    PS|    6|
|                    AM|    1|
|                    DR|   39|
|                    JF|    1|
|                     F|    1|
|                    BP|  138|
|                     Q|    7|
|                    NX|    1|
|                    JC|    6|
|                    NA|    6|
|                    HV|    5|
|                    BX|    1|
|                    VA|    1|
|                     E|    1|
+----------------------+-----+
only showing top 20 rows



In [22]:
df_s.groupBy(split(df_s['Sınıflama'], " ")[0].alias("group")).count().orderBy("group").collect()

[Row(group='AM', count=1),
 Row(group='B', count=6),
 Row(group='BD', count=2),
 Row(group='BF', count=9),
 Row(group='BJ', count=4),
 Row(group='BL', count=7),
 Row(group='BP', count=138),
 Row(group='BQ', count=1),
 Row(group='BS', count=3),
 Row(group='BT', count=1),
 Row(group='BV', count=1),
 Row(group='BX', count=1),
 Row(group='CB', count=3),
 Row(group='CC', count=1),
 Row(group='D', count=13),
 Row(group='DD', count=2),
 Row(group='DF', count=4),
 Row(group='DG', count=2),
 Row(group='DK', count=4),
 Row(group='DR', count=39),
 Row(group='DS', count=19),
 Row(group='DT', count=1),
 Row(group='E', count=1),
 Row(group='F', count=1),
 Row(group='GE', count=1),
 Row(group='GN', count=2),
 Row(group='GR', count=4),
 Row(group='GV', count=2),
 Row(group='H', count=4),
 Row(group='HA', count=1),
 Row(group='HB', count=8),
 Row(group='HC', count=6),
 Row(group='HD', count=6),
 Row(group='HF', count=11),
 Row(group='HJ', count=2),
 Row(group='HM', count=8),
 Row(group='HN', count=2),


In [23]:
df1.fillna({'Ödünç Sayısı' : 0.0}).show()

+-----------------+--------------------+--------------------+---+--------------------+------------+
|        Sınıflama|            Eser Adı|               Yazar|Dil|     Konu Başlıkları|Ödünç Sayısı|
+-----------------+--------------------+--------------------+---+--------------------+------------+
|      AM 7/.M8713|Museum frictions ...|                    |eng|Museums--Social A...|         0.0|
|   B 105 .I49/T34|Tahayyül gücünü y...|                    |tur|Imagination (Phil...|         0.0|
|  B 3279 .H48/D36|Bir yol var : Min...|        Damcı, Taner|tur|Ontology
Stress M...|         0.0|
|  B 415 .A9/A7519|Psikoloji şerhi =...|            İbn Rüşd|tur|Aristotle
Aristot...|         2.0|
| B 5074 .S544/K39|An examination of...|      Kaya, Vefa Can|eng|Özel, İsmet, 1944...|         0.0|
| B 5074 .S544/K39|An examination of...|      Kaya, Vefa Can|eng|Özel, İsmet, 1944...|         0.0|
| B 753 .I53/R5320|Ruhun uyanışı, ya...|İbn Ṭufeyl, Muḥam...|tur|Philosophy, Islam...|         0.0|


In [24]:
df.groupBy(df['Sınıflama']).agg(collect_list('Konu Başlıkları').alias('gruplanmış konu başlıkları')).show()

+------------------+--------------------------+
|         Sınıflama|gruplanmış konu başlıkları|
+------------------+--------------------------+
|   BF 635 .G43/C34|      [Psychology, Path...|
|       D 16/.H3519|      [History--Methodo...|
|   PL 248 .T67/D47|      [Short Stories, T...|
| BP 189.7 .C58/O57|      [Jalal Al-Din Rum...|
|    NK 3636.5/.L57|      [Hilye-i Şerif--E...|
|     PN 1997/.F357|      [Crime Films
Dram...|
|   DR 479 .E85/P68|      [Europe--Relation...|
|HQ 1726.7 .A37/T67|      [Women--Turkey
Hu...|
| HQ 767.9 .C36/R47|      [Child Developmen...|
|  NA 1363 .K83/K66|      [Architecture, Se...|
|   PN 6757 .K54/C3|      [Castro, Fidel, 1...|
|     BP 63 .T8/B37|      [Islam--Turkey--2...|
|       HV 42/.B676|      [Endowments--Hist...|
|        Q 335/.R86|      [Artificial Intel...|
|   BP 130.1/.M3719|      [Qur'an
Kur’an-ı ...|
| DR 438.5 .A36/F45|      [Turkey--History-...|
|   PL 248 .Y55/H35|      [Turkish Literatu...|
| BF 692.2 .S65/B36|      [Sex Differenc

In [25]:
df.groupBy(df['Sınıflama']).agg(collect_list('Konu Başlıkları'), collect_list('Yazar')).show()

+------------------+-----------------------------+--------------------+
|         Sınıflama|collect_list(Konu Başlıkları)| collect_list(Yazar)|
+------------------+-----------------------------+--------------------+
|   BF 635 .G43/C34|         [Psychology, Path...|     [Geçtan, Engin]|
|       D 16/.H3519|         [History--Methodo...|[Halkin, Léon-Ern...|
|   PL 248 .T67/D47|         [Short Stories, T...|   [Toprak, Nurcan ]|
| BP 189.7 .C58/O57|         [Jalal Al-Din Rum...|  [Çıtlak, M. Fatih]|
|    NK 3636.5/.L57|         [Hilye-i Şerif--E...|                  []|
|     PN 1997/.F357|         [Crime Films
Dram...|                  []|
|   DR 479 .E85/P68|         [Europe--Relation...| [Poumarede, Geraud]|
|HQ 1726.7 .A37/T67|         [Women--Turkey
Hu...|     [Aksoy, Nurgül]|
| HQ 767.9 .C36/R47|         [Child Developmen...|    [Canan, İbrahim]|
|  NA 1363 .K83/K66|         [Architecture, Se...|   [Küçükdağ, Yusuf]|
|   PN 6757 .K54/C3|         [Castro, Fidel, 1...|  [Kleist, Rei

In [29]:
df.filter(df['Ödünç Sayısı'] != 0.0).select("Eser Adı").collect()

[Row(Eser Adı="Psikoloji şerhi =  Telhisu kitabi'n-nefs"),
 Row(Eser Adı='Çağımızın sorunları üstüne düşünceler'),
 Row(Eser Adı="Heidegger'in kulübesi"),
 Row(Eser Adı='Bana bilgiçlik taslayan adamlar = Men explain things to me'),
 Row(Eser Adı='Sahip olmak ya da olmak'),
 Row(Eser Adı='Geliştiren anne baba'),
 Row(Eser Adı='Research design and methods : a process approach'),
 Row(Eser Adı='Hayriyye-i Nabi : inceleme-metin'),
 Row(Eser Adı='Din psikolojisi'),
 Row(Eser Adı="Mitoloji Kur'an kıssaları ve tarihi gerçeklik"),
 Row(Eser Adı='Hadislerin dili : ilk hadis belgesi Hemmam’ın Sahifesi : tertip, terceme, yorum'),
 Row(Eser Adı='Güncel Dini Meseleler İstişare Toplantısı'),
 Row(Eser Adı='Neoliberal İslamcılık 1980-2015 : İslamcıların dünya sistemine entegrasyonu'),
 Row(Eser Adı='Cumhuriyet Türkiyesi’nde bir mesele olarak İslam'),
 Row(Eser Adı='Cumhuriyet Türkiyesi’nde bir mesele olarak İslam'),
 Row(Eser Adı='Kapitalizme eklemlenme : dindar orta sınıfta tüketim kültürü'),
 Row(E